# SPLADE for Portuguese

Inspired by https://github.com/naver/splade

In [1]:
!nvidia-smi

Thu Jun 22 21:03:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
user = "monique" # monique
if user == "monique":
    main_dir = '/content/gdrive/MyDrive/Unicamp-projeto-final/'
else:
    main_dir = '/content/gdrive/MyDrive/Unicamp/IA368-DD/'

## Libraries installation

In [4]:
%%shell
pip install pytrec_eval
pip install git+https://github.com/naver/splade.git -q
pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293472 sha256=7f103a0b37ded41864065451b60f3aaaa0354ba480bc6480073d34bd87a28796
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 70.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
import pytrec_eval
pytrec_eval.supported_measures

{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}

## Copying data files to the expected data structure

In [6]:
!git clone https://github.com/naver/splade.git

Cloning into 'splade'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 667 (delta 109), reused 117 (delta 71), pack-reused 467
Receiving objects: 100% (667/667), 3.13 MiB | 12.89 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [7]:
import shutil
import os

shutil.copyfile(f"{main_dir}Projeto Final/data.zip", "/content/data.zip")
!unzip data.zip

Archive:  data.zip
   creating: content/data/
   creating: content/data/m_marco/
  inflating: content/data/m_marco/queries_train.tsv  
   creating: content/data/m_marco/.ipynb_checkpoints/
  inflating: content/data/m_marco/triplets.tsv  
  inflating: content/data/m_marco/queries_dev.full.tsv  
  inflating: content/data/m_marco/queries_dev.tsv  
  inflating: content/data/m_marco/corpus.tsv  
   creating: content/data/m_robust/
  inflating: content/data/m_robust/qrels.robust04.txt  
   creating: content/data/m_robust/.ipynb_checkpoints/
  inflating: content/data/m_robust/queries.tsv  
  inflating: content/data/m_robust/corpus.tsv  


In [8]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/

In [9]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/data/pt.yaml" splade/conf/train/data

### Copying full collection

For indexing we'll validate the model on mROBUST dataset.  So, its corpus should be used as full collection.

In [10]:
!mkdir -p splade/data/pt/full_collection

Preprocesses mROBUST dataset in order to replace alphanumerical IDs with integer ones.

In [11]:
import pandas as pd

#df_mrobust_corpus = pd.read_csv("content/data/m_robust/corpus.tsv", sep='\t', header=None)
#df_mrobust_corpus.head()
map_id_to_text = dict()

with open("content/data/m_robust/corpus.tsv", 'r') as file:
  for line in file:
    fields = line.split('\t')
    id = fields[0]
    text = fields[1]
    map_id_to_text[id] = text

In [12]:
doc_ids = map_id_to_text.keys()

In [19]:
map_id_to_int = {value:index for index, value in enumerate(doc_ids)}


In [20]:
map_id_to_int['FT931-12718']

654

In [21]:
len(map_id_to_text)

528032

In [22]:
new_map_id_to_text = {map_id_to_int[id]: text for id, text in map_id_to_text.items()}

In [23]:
len(new_map_id_to_text)

528032

In [24]:
with open('new_corpus.tsv', 'w') as file:
  for id, text in new_map_id_to_text.items():
    file.write(f"{id}\t{text}")

In [25]:
!head content/data/m_robust/corpus.tsv

FT931-16655	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aum

In [26]:
!head new_corpus.tsv

0	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aumentando su

In [27]:
!wc -l content/data/m_robust/corpus.tsv

528032 content/data/m_robust/corpus.tsv


In [28]:
!wc -l new_corpus.tsv

528032 new_corpus.tsv


In [17]:
!cp new_corpus.tsv splade/data/pt/full_collection/raw.tsv

cp: cannot stat 'new_corpus.tsv': No such file or directory


In [18]:
!head splade/data/pt/full_collection/raw.tsv

head: cannot open 'splade/data/pt/full_collection/raw.tsv' for reading: No such file or directory


In [29]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/index/pt.yaml" splade/conf/index

### Retrieval and evaluation configuration

Here, we'll configure retrieval and evaluation for mROBUST dataset.

In [30]:
!mkdir -p splade/data/pt/dev_queries

In [31]:
!cp content/data/m_robust/queries.tsv splade/data/pt/dev_queries/raw.tsv

Converts QREL information to JSON fomat.

In [61]:
#TODO: Devo fazer o dump apenas dos que têm relevância 1?
from collections import defaultdict
import json

qrel = defaultdict(dict)

with open("content/data/m_robust/qrels.robust04.txt", 'r') as file:
  for line in file:
    fields = line.split()
    q_id = fields[0]
    if fields[2] in map_id_to_int:
      doc_id = map_id_to_int[fields[2]]
      rel = fields[3]
      qrel[q_id][doc_id] = int(rel)

with open('splade/data/pt/dev_qrel.json', 'w') as file:
    json.dump(qrel, file)

In [44]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/retrieve_evaluate/pt.yaml" splade/conf/retrieve_evaluate

### Flops estimation

We'll use the same test set (mRobust) to estimate flops.

In [34]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/flops/pt.yaml" splade/conf/flops

## Indexing and retrieving

In [35]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/model/splade_bertimbau_base.yaml" splade/conf/train/model/

In [36]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/config/splade_pt.yaml" splade/conf/train/config

In [ ]:
shutil.copyfile(f"{main_dir}Projeto Final/experiments_10m.zip", "/content/experiments.zip")

In [38]:
!unzip /content/experiments.zip
!mv /content/content/splade/experiments /content/experiments

Archive:  /content/experiments.zip
   creating: content/splade/experiments/
   creating: content/splade/experiments/pt/
   creating: content/splade/experiments/pt/checkpoint/
  inflating: content/splade/experiments/pt/checkpoint/training_perf.txt  
   creating: content/splade/experiments/pt/checkpoint/model/
  inflating: content/splade/experiments/pt/checkpoint/model/special_tokens_map.json  
  inflating: content/splade/experiments/pt/checkpoint/model/vocab.txt  
  inflating: content/splade/experiments/pt/checkpoint/model/model.tar  
  inflating: content/splade/experiments/pt/checkpoint/model/pytorch_model.bin  
  inflating: content/splade/experiments/pt/checkpoint/model/tokenizer.json  
  inflating: content/splade/experiments/pt/checkpoint/model/config.json  
  inflating: content/splade/experiments/pt/checkpoint/model/tokenizer_config.json  
   creating: content/splade/experiments/pt/checkpoint/val_full_ranking/
  inflating: content/splade/experiments/pt/checkpoint/val_full_ranking/ru

In [57]:
!cp -r experiments/pt splade/experiments

In [58]:
!rm -r splade/experiments/checkpoint

In [59]:
!rm -r splade/experiments/pt/pt

In [40]:
!ls experiments/pt/checkpoint/

config.yaml  model_ckpt   training_perf.txt  validation_perf.txt
model	     tensorboard  val_full_ranking


In [41]:
shutil.copyfile(f"{main_dir}Projeto Final/index_10m.zip", "/content/index.zip")

'/content/index.zip'

In [42]:
!unzip /content/index.zip
#!mv /content/content/splade/experiments /content/experiments

Archive:  /content/index.zip
   creating: splade/experiments/pt/index/
  inflating: splade/experiments/pt/index/doc_ids.pkl  
  inflating: splade/experiments/pt/index/array_index.h5py  
  inflating: splade/experiments/pt/index/index_dist.json  
 extracting: splade/experiments/pt/index/index_stats.json  


In [52]:
!mkdir -p experiments/pt/out/other_dataset


In [63]:
%%shell
cd splade/
export PYTHONPATH=$PYTHONPATH:$(pwd)
export SPLADE_CONFIG_NAME="config_splade_pt.yaml"
python3 -m splade.retrieve \
  config.checkpoint_dir=experiments/pt/checkpoint \
  config.index_dir=experiments/pt/index \
  config.out_dir=experiments/pt/out

/content/splade/splade/evaluate.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
/content/splade/splade/retrieve.py:13: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_splade_pt': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1

In [64]:
!zip -r out_10m.zip splade/experiments/pt/out

  adding: splade/experiments/pt/out/ (stored 0%)
  adding: splade/experiments/pt/out/other_dataset/ (stored 0%)
  adding: splade/experiments/pt/out/other_dataset/stats/ (stored 0%)
  adding: splade/experiments/pt/out/other_dataset/stats/q_stats.json (stored 0%)
  adding: splade/experiments/pt/out/other_dataset/run.json (deflated 61%)
  adding: splade/experiments/pt/out/other_dataset/perf.json (deflated 59%)
  adding: splade/experiments/pt/out/perf_all_datasets.json (deflated 58%)


In [65]:
shutil.copyfile("out_10m.zip", f"{main_dir}Projeto Final/out_10m.zip",)

'/content/gdrive/MyDrive/Unicamp-projeto-final/Projeto Final/out_10m.zip'